In [19]:
a=torch.zeros([8,2], dtype=torch.float32)

In [5]:
coordinates

[{'x_min': 522, 'y_min': 138, 'x_max': 591, 'y_max': 393},
 {'x_min': 943, 'y_min': 466, 'x_max': 1013, 'y_max': 720}]

In [7]:
import xml.etree.ElementTree as ET
def bboxParser(fileName):
    root = ET.parse(fileName + '.xml')
    coordinates = []
    for obj in root.iter('HRSC_Object'):
        coordinate = {}
        coordinate['x_min'] = int(obj.find('box_xmin').text)
        coordinate['y_min'] = int(obj.find('box_ymin').text)
        coordinate['x_max'] = int(obj.find('box_xmax').text)
        coordinate['y_max'] = int(obj.find('box_ymax').text)
        coordinates.append(coordinate)
        
    return int(root.find('Img_SizeWidth').text), int(root.find('Img_SizeHeight').text), coordinates

In [8]:
w,h,a = bboxParser('test')

In [6]:
a

[{'x_min': 522, 'y_min': 138, 'x_max': 591, 'y_max': 393},
 {'x_min': 943, 'y_min': 466, 'x_max': 1013, 'y_max': 720}]

In [5]:
import torch
def weight(shape):
    w = torch.empty(shape)
    initial = torch.nn.init.trunc_normal_(w)
    return initial

In [6]:
w = weight((3,5))

In [7]:
w

tensor([[-0.0562,  0.1950,  0.2111, -0.7356,  0.4723],
        [-0.5127,  0.7964,  0.8249, -1.7378,  0.3028],
        [ 0.2230, -0.6746, -1.2672, -1.6923, -1.3758]])

In [ ]:
import cv2 as cv
import numpy as np
max_h = 0
max_w = 0
for i in range(1,1681):
    name = './current_data/AllImages/1' + str(i).zfill(8) + '.bmp'
    img = cv.imread(name)
    if img is None:
        pass
    else:
        img_h, img_w, ch = img.shape
        print(name)
        if img_h > max_h:
            max_h = img_h
        if img_w > max_w:
            max_w = img_w
print('max_h: %d, max_w: %d' % (max_h, max_w))

In [6]:
import cv2 as cv
import numpy as np
new_h = 844
new_w = 1238
for i in range(1,1681):
    name = './current_data/AllImages/1' + str(i).zfill(8) + '.bmp'
    img = cv.imread(name)
    if img is None:
        pass
    else:
        img_h, img_w, ch = img.shape
        padding = np.full((new_h,new_w,3),(34,47,32), dtype=np.uint8)
        padding[0:img_h, 0:img_w] = img
        cv.imwrite('./pad/'+str(i)+'.bmp', padding)
